## Import Dependencies

In [1]:
import csv
from pathlib import Path
import pandas as pd
import glob
from datetime import datetime, timedelta
import numpy as np
import pandas_profiling as pp

# Prepare the Processed Acquisition file

In [2]:
acq_df = pd.read_csv('D:\FMAcqProcessed.csv')

In [3]:
#Dropping columns we don't want
acq_df.drop(['Unnamed: 0','origCLTV'],1,inplace=True)

In [4]:
acq_df['origDate'] = pd.to_datetime(acq_df['origDate'], format='%Y/%m/%d')
acq_df['firstPmtDate'] = pd.to_datetime(acq_df['firstPmtDate'], format='%Y/%m/%d')

In [5]:
# sellerNames = acq_df['sellerName'].unique()
# statesList = acq_df['propState'].unique()

In [6]:
# # Make copy of the column to map from strings to integers
# acq_df['bankNumber'] = acq_df['sellerName']
# # Coerce the new column to category type
# acq_df['bankNumber'] = acq_df['bankNumber'].astype('category')
# # Convert the new column to integers for each unique string value
# acq_df['bankNumber'] = acq_df['bankNumber'].cat.codes
# # Capture the mapping to a dictionary
# sellerNameMap = pd.Series(acq_df.sellerName.values,index=acq_df.bankNumber).to_dict()
# # Drop original column leaving only the integer column for ML purposes
# acq_df.drop(columns=['sellerName'],inplace=True)
# print(sorted(sellerNameMap.items()))

In [7]:
# # Make copy of the column to map from strings to integers
# acq_df['stateNumber'] = acq_df['propState']
# # Coerce the new column to category type
# acq_df['stateNumber'] = acq_df['stateNumber'].astype('category')
# # Convert the new column to integers for each unique string value
# acq_df['stateNumber'] = acq_df['stateNumber'].cat.codes
# # Capture the mapping to a dictionary
# statePropMap = pd.Series(acq_df.propState.values,index=acq_df.stateNumber).to_dict()
# # Drop original column leaving only the integer column for ML purposes
# acq_df.drop(columns=['propState'],inplace=True)
# print(sorted(statePropMap.items()))

In [8]:
acq_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10464607 entries, 0 to 10464606
Data columns (total 19 columns):
loanIdentifier      int64
origChannel         int64
sellerName          object
origIntRate         float64
origUPB             int64
origDate            datetime64[ns]
firstPmtDate        datetime64[ns]
origLTV             float64
numBorrowers        float64
origDebtIncRatio    float64
borrCreditScore     float64
loanPurp            int64
propState           object
zipCode             int64
pMIperct            float64
mortInsType         float64
bestCreditScore     float64
worstCreditScore    float64
avgCreditScore      float64
dtypes: datetime64[ns](2), float64(10), int64(5), object(2)
memory usage: 1.5+ GB


In [9]:
acq_df.head()

,loanIdentifier,origChannel,sellerName,origIntRate,origUPB,origDate,firstPmtDate,origLTV,numBorrowers,origDebtIncRatio,borrCreditScore,loanPurp,propState,zipCode,pMIperct,mortInsType,bestCreditScore,worstCreditScore,avgCreditScore
0,100007365142,1,"JPMORGAN CHASE BANK, NA",8.000,75000,1999-12-01,2000-02-01,79.0,1.0,62.0,763.0,2,PA,173,0.0,0.0,763.0,763.0,763.0
1,100094030068,3,"CITIMORTGAGE, INC.",7.750,152000,1999-11-01,2000-01-01,85.0,1.0,41.0,604.0,2,NV,891,25.0,1.0,604.0,604.0,604.0
2,100118647578,2,"JPMORGAN CHASE BANK, NA",7.750,106000,1999-11-01,2000-01-01,80.0,2.0,27.0,652.0,2,OR,972,0.0,0.0,696.0,652.0,674.0
3,100122555751,1,"GMAC MORTGAGE, LLC",8.250,191000,2000-02-01,2000-04-01,78.0,1.0,52.0,676.0,1,CA,921,0.0,0.0,676.0,676.0,676.0
4,100123116925,3,"BANK OF AMERICA, N.A.",7.875,240000,1999-12-01,2000-02-01,74.0,3.0,40.0,718.0,2,CA,940,0.0,0.0,718.0,685.0,701.5


# Prepare the Processed Performance file

In [10]:
perf_df = pd.read_csv('D:\FMPerf2000Processed.csv')

In [11]:
perf_df['rptPeriod'] = pd.to_datetime(perf_df['rptPeriod'], format='%Y/%m/%d')
perf_df['lastPdInstDate'] = pd.to_datetime(perf_df['lastPdInstDate'], format='%Y/%m/%d')
perf_df['zeroBalDate'] = pd.to_datetime(perf_df['zeroBalDate'], format='%Y/%m/%d')
perf_df['forecloDate'] = pd.to_datetime(perf_df['forecloDate'], format='%Y/%m/%d')
perf_df['dispDate'] = pd.to_datetime(perf_df['dispDate'], format='%Y/%m/%d')

In [12]:
perf_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191753 entries, 0 to 191752
Data columns (total 29 columns):
id                            191753 non-null int64
rptPeriod                     191753 non-null datetime64[ns]
currIntRate                   191753 non-null float64
currUPB                       191742 non-null float64
loanAge                       191753 non-null int64
monMatur                      191486 non-null float64
mSA                           191753 non-null int64
zeroBalCode                   191753 non-null float64
zeroBalDate                   191753 non-null datetime64[ns]
lastPdInstDate                15210 non-null datetime64[ns]
forecloDate                   14975 non-null datetime64[ns]
dispDate                      15083 non-null datetime64[ns]
forecloCost                   14129 non-null float64
propRepCost                   12250 non-null float64
recovCosts                    5444 non-null float64
miscCost                      12950 non-null float64
hold

## Get Freddie Mac 30 yr weekly rate

In [13]:
# import training_data via pandas module
fmac30 = pd.read_csv("download/FMAC-30US.csv", low_memory=False)

In [14]:
fmac30.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2565 entries, 0 to 2564
Data columns (total 2 columns):
Date     2565 non-null object
Value    2565 non-null float64
dtypes: float64(1), object(1)
memory usage: 40.2+ KB


In [15]:
fmac30['Date'] = pd.to_datetime(fmac30['Date'])

In [16]:
fmac30 = fmac30.set_index('Date').resample('D').ffill().reset_index()

In [17]:
fmac30 = fmac30.rename(columns={'Date':'origDate','Value':'fmacRate'})

## Get 10 yr Treasury daily rate

In [18]:
# import training_data via pandas module
fred10 = pd.read_csv("download/FRED-DGS10.csv", low_memory=False)

In [19]:
fred10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14575 entries, 0 to 14574
Data columns (total 2 columns):
Date     14575 non-null object
Value    14575 non-null float64
dtypes: float64(1), object(1)
memory usage: 227.8+ KB


In [20]:
fred10['Date'] = pd.to_datetime(fred10['Date'])

In [21]:
fred10 = fred10.set_index('Date').resample('D').ffill().reset_index()

In [22]:
fred10 = fred10.rename(columns={'Date':'origDate','Value':'fredRate'})

## Merge data

In [23]:
acq_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10464607 entries, 0 to 10464606
Data columns (total 19 columns):
loanIdentifier      int64
origChannel         int64
sellerName          object
origIntRate         float64
origUPB             int64
origDate            datetime64[ns]
firstPmtDate        datetime64[ns]
origLTV             float64
numBorrowers        float64
origDebtIncRatio    float64
borrCreditScore     float64
loanPurp            int64
propState           object
zipCode             int64
pMIperct            float64
mortInsType         float64
bestCreditScore     float64
worstCreditScore    float64
avgCreditScore      float64
dtypes: datetime64[ns](2), float64(10), int64(5), object(2)
memory usage: 1.5+ GB


In [24]:
# Merge data
merged = pd.merge(acq_df, fmac30, on=["origDate"])
merged.shape

(10464607, 20)

In [25]:
# Merge data
merged = pd.merge(merged, fred10, on=["origDate"])
merged.shape

(10464607, 21)

In [26]:
merged.rename(columns={'loanIdentifier':'id'}, inplace=True )

In [27]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10464607 entries, 0 to 10464606
Data columns (total 21 columns):
id                  int64
origChannel         int64
sellerName          object
origIntRate         float64
origUPB             int64
origDate            datetime64[ns]
firstPmtDate        datetime64[ns]
origLTV             float64
numBorrowers        float64
origDebtIncRatio    float64
borrCreditScore     float64
loanPurp            int64
propState           object
zipCode             int64
pMIperct            float64
mortInsType         float64
bestCreditScore     float64
worstCreditScore    float64
avgCreditScore      float64
fmacRate            float64
fredRate            float64
dtypes: datetime64[ns](2), float64(12), int64(5), object(2)
memory usage: 1.7+ GB


In [28]:
# Merge data
merged = pd.merge(merged, perf_df, on=["id"])
merged.shape

(37822, 49)

In [29]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37822 entries, 0 to 37821
Data columns (total 49 columns):
id                            37822 non-null int64
origChannel                   37822 non-null int64
sellerName                    37822 non-null object
origIntRate                   37822 non-null float64
origUPB                       37822 non-null int64
origDate                      37822 non-null datetime64[ns]
firstPmtDate                  37822 non-null datetime64[ns]
origLTV                       37822 non-null float64
numBorrowers                  37822 non-null float64
origDebtIncRatio              37822 non-null float64
borrCreditScore               37822 non-null float64
loanPurp                      37822 non-null int64
propState                     37822 non-null object
zipCode                       37822 non-null int64
pMIperct                      37822 non-null float64
mortInsType                   37822 non-null float64
bestCreditScore               37822 non-n

In [30]:
merged.to_csv('data/FM_Acq_Perf_FMAC_2000_EDA.csv')